In [1]:
import time
import numpy as np
from gep_config_parser import *
from data_wrangling import dataframe_to_dict
import pyomo.environ as pyo

from gep_main import run_model, prep_data, run_model_no_bounds
from gep_problem import GEPProblem

import torch

DTYPE = torch.float64

torch.set_default_dtype(DTYPE)

# DEVICE = (
#     torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
# )

DEVICE = "cpu"


Parsing the config file
Initializing the solver
Using Gurobi


In [2]:

CONFIG_FILE_NAME        = "config.toml"
VISUALIZATION_FILE_NAME = "visualization.toml"

HIGHS  = "HiGHS"
GUROBI = "Gurobi"

SAMPLE_DURATION = 12 # 12 hours

## Step 0: Activate environment - ensure consistency accross computers
# print("Reading the data")
# print("Activating the environment")


In [3]:

## Step 1: parse the input data
print("Parsing the config file")

data = parse_config(CONFIG_FILE_NAME)
experiment = data["experiment"]
outputs_config = data["outputs_config"]

print("Initializing the solver")
optimizer_name = data["optimizer_config"]["solver"]

# Determine the optimizer
if optimizer_name == HIGHS:
    raise NotImplementedError(f"{optimizer_name}: Not implemented")
elif optimizer_name == GUROBI:
    
    print(f"Using {GUROBI}")
    optimizer = "gurobi_direct"
else:
    raise ValueError(f"{optimizer_name}: Not implemented")

Parsing the config file
Initializing the solver
Using Gurobi


In [ ]:
for i, experiment_instance in enumerate(experiment["experiments"]):
    # Setup output dataframe
    df_res = pd.DataFrame(columns=["setup_time", "presolve_time", "barrier_time", "crossover_time", "restore_time", "objective_value"])

    input_T, input_N, input_G, input_L, input_pDemand, input_pGenAva, input_pVOLL, input_pWeight, input_pRamping, input_pInvCost, input_pVarCost, input_pUnitCap, input_pExpCap, input_pImpCap = prep_data(experiment_instance)

    T_ranges = [range(i, i + SAMPLE_DURATION, 1) for i in range(1, len(input_T), SAMPLE_DURATION)]
    objective_values = []
    times = []
    models = []
    solvers = []
    for t in T_ranges[:1]:
        # Run one experiment for j repeats
        model, solver, time_taken = run_model(experiment_instance, t, input_N, input_G, input_L, input_pDemand, input_pGenAva, input_pVOLL, input_pWeight, input_pRamping, input_pInvCost, input_pVarCost, input_pUnitCap, input_pExpCap, input_pImpCap)
        models.append(model)
        solvers.append(solver)

Wrangling the input data
Populating the model
Adding model variables
Formulating the objective
Adding model constraints
Solving the optimization problem
Set parameter OutputFlag to value 1
Set parameter LogFile to value "outputs/Gurobi/output.txt"
Set parameter Crossover to value 0
Set parameter FeasibilityTol to value 1e-09
Set parameter QCPDual to value 1
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 22.3.0 22D49)

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Non-default parameters:
FeasibilityTol  1e-09
Method  2
Crossover  0
QCPDual  1

Optimize a model with 140 rows, 113 columns and 467 nonzeros
Model fingerprint: 0x93c00021
Coefficient statistics:
  Matrix range     [7e-02, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [2e+03, 3e+04]
  RHS range        [4e+03, 3e+04]
Presolve removed 126 rows and 104 columns
Presolve time: 0.00s
Presolved: 14 rows, 9 columns, 42 nonzeros
Ordering time

In [5]:
def get_variable_values_as_list(var):
    # Check if the variable is indexed
    if var.is_indexed():
        return [var[idx].value for idx in var]
    else:
        # For scalar variables
        return [var.value]

# Function to get constraint values as a list
def get_constraint_values_as_list(constraint):
    constraint_values = []
    for idx in constraint:
        # Evaluate the constraint body
        constraint_expr_value = pyo.value(constraint[idx].body)
        constraint_values.append(constraint_expr_value)
    return constraint_values

def get_variable_bounds(var):
    lower_bounds = []
    upper_bounds = []
    
    # Check if the variable is indexed
    if var.is_indexed():
        # Iterate through the indices of the variable
        for idx in var:
            lower_bounds.append(var[idx].lb)  # Get lower bound for each index
            upper_bounds.append(var[idx].ub)  # Get upper bound for each index
    else:
        # For scalar variables
        lower_bounds.append(var.lb)  # Get lower bound of scalar variable
        upper_bounds.append(var.ub)  # Get upper bound of scalar variable

    return lower_bounds, upper_bounds


# !!!! 
### When using MPS, calculations are done in float32. This introduces errors of the order e-3. 

In [12]:
#T, N, G, L, pDemand, pGenAva, pVOLL, pWeight, pRamping, pInvCost, pVarCost, pUnitCap, pExpCap, pImpCap
data = GEPProblem(input_T, input_N, input_G, input_L, input_pDemand, input_pGenAva, input_pVOLL, input_pWeight, input_pRamping, input_pInvCost, input_pVarCost, input_pUnitCap, input_pExpCap, input_pImpCap, sample_duration=SAMPLE_DURATION, shuffle=False)

# X contains [pGenAva, pDemand]
all_X = torch.concat([data.trainX, data.validX, data.testX])

Size of train set: 584
Size of val set: 73
Size of test set: 73
Size of mu: 285
Size of lambda: 36
Number of variables (size of y): 111
Number of inputs (size of X): 72


In [14]:
print("### Testing consistency of g_x(y) ###")

# Prepare batched inputs
X = all_X  # Adjust for batched input
Y_list = [
    torch.tensor(
        [[*get_variable_values_as_list(model.vGenProd),
          *get_variable_values_as_list(model.vLineFlow),
          *get_variable_values_as_list(model.vLossLoad),
          *get_variable_values_as_list(model.vGenInv)]],
        device=DEVICE
    ) for model in models
]
Y = torch.cat(Y_list, dim=0)  # Batch all Y tensors

# Compute g in a batched way
g_batch = data.ineq_resid(X, Y, scaled_X=True).squeeze(1).tolist()

# Process constraints in batch
eMaxProd_idx = len(input_G) * 12
eMaxProd_PDL = [g[:eMaxProd_idx] for g in g_batch]
eMaxProd_LP_list = [get_constraint_values_as_list(model.eMaxProd) for model in models]

# Check eMaxProd consistency
for g_pdl, g_lp in zip(eMaxProd_PDL, eMaxProd_LP_list):
    assert np.allclose(g_pdl, g_lp, atol=1e-9)

# Process other constraints in batch
for i, (g, model) in enumerate(zip(g_batch, models)):
    print(i)
    eLineFlow_idx_lb = eMaxProd_idx + len(input_L) * 12
    eLineFlow_idx_ub = eLineFlow_idx_lb + len(input_L) * 12

    eLineFlow_PDL_lb = np.array(g[eMaxProd_idx:eLineFlow_idx_lb], dtype=np.float64)
    eLineFlow_PDL_ub = np.array(g[eLineFlow_idx_lb:eLineFlow_idx_ub], dtype=np.float64)

    eLineFlow_LP_lb, eLineFlow_LP_ub = (
        np.array(get_variable_bounds(model.vLineFlow)[0], dtype=np.float64),
        np.array(get_variable_bounds(model.vLineFlow)[1], dtype=np.float64),
    )
    vLineFlow = np.array(get_variable_values_as_list(model.vLineFlow), dtype=np.float64)

    assert np.allclose(eLineFlow_PDL_lb + vLineFlow, eLineFlow_LP_lb, atol=1e-9)
    assert np.allclose(-1 * (eLineFlow_PDL_ub - vLineFlow), eLineFlow_LP_ub, atol=1e-9)

    # Continue with other constraint tests similarly...
    eRampingDown_idx = eLineFlow_idx_ub + len(input_G) * (12 - 1)
    eRampingDown_PDL = g[eLineFlow_idx_ub:eRampingDown_idx]
    eRampingDown_LP = get_constraint_values_as_list(model.eRampingDown)

    assert np.allclose(np.array(eRampingDown_PDL), np.array(eRampingDown_LP), atol=1e-9)

    eRampingUp_idx = eRampingDown_idx + len(input_G) * (12 - 1)
    eRampingUp_PDL = g[eRampingDown_idx:eRampingUp_idx]
    eRampingUp_LP = get_constraint_values_as_list(model.eRampingUp)

    assert np.allclose(np.array(eRampingUp_PDL), np.array(eRampingUp_LP), atol=1e-9)

    eGenProdPositive_idx = eRampingUp_idx + len(input_G) * 12
    eGenProdPositive_PDL = g[eRampingUp_idx:eGenProdPositive_idx]
    vGenProd = get_variable_values_as_list(model.vGenProd)

    assert np.allclose(np.array(eGenProdPositive_PDL), -1 * np.array(vGenProd), atol=1e-9)

### Testing consistency of g_x(y) ###
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267


In [15]:
print("### Testing consistency of h_x(y) ###")

# Prepare batched inputs
X = all_X  # Adjust for batched input
Y_list = [
    torch.tensor(
        [[*get_variable_values_as_list(model.vGenProd),
          *get_variable_values_as_list(model.vLineFlow),
          *get_variable_values_as_list(model.vLossLoad),
          *get_variable_values_as_list(model.vGenInv)]],
        device=DEVICE
    ) for model in models
]
Y = torch.cat(Y_list, dim=0)  # Batch all Y tensors

# Compute h in a batched way
h_batch = data.eq_resid(X, Y).squeeze(1).tolist()

# Prepare eNodeBal and pDemand for all models
eNodeBal_LP_list = [get_constraint_values_as_list(model.eNodeBal) for model in models]
pDemand_list = [
    data.pDemand_tensor[:, i * SAMPLE_DURATION:(i + 1) * SAMPLE_DURATION].flatten().tolist()
    for i in range(all_X.shape[0])
]

# Check eNodeBal consistency in batch
for h, eNodeBal_LP, pDemand in zip(h_batch, eNodeBal_LP_list, pDemand_list):
    assert np.allclose(
        np.array(h), 
        np.array(eNodeBal_LP) - np.array(pDemand), 
        atol=1e-8
    ), (
        f"Assertion failed: eNodeBal_PDL != eNodeBal_LP - pDemand\n"
        f"  h (PDL): {h}\n"
        f"  eNodeBal_LP: {eNodeBal_LP}\n"
        f"  pDemand: {pDemand}"
    )

### Testing consistency of h_x(y) ###


In [16]:
print("### Testing consistency of f_x(y) ###")

# Prepare batched inputs for Y
Y_list = [
    torch.tensor(
        [[*get_variable_values_as_list(model.vGenProd),
          *get_variable_values_as_list(model.vLineFlow),
          *get_variable_values_as_list(model.vLossLoad),
          *get_variable_values_as_list(model.vGenInv)]],
        device=DEVICE
    ) for model in models
]
Y = torch.cat(Y_list, dim=0)  # Batch all Y tensors

# Compute objective function values for all models
obj_LP_list = [model.obj() for model in models]
obj_PDL_list = data.obj_fn(Y).tolist()

# Check objective function consistency in batch
for obj_LP, obj_PDL in zip(obj_LP_list, obj_PDL_list):
    assert abs((obj_LP - obj_PDL) / obj_LP) < 1e-9, (
        f"Assertion failed: Objective mismatch\n"
        f"  obj_LP: {obj_LP}\n"
        f"  obj_PDL: {obj_PDL}\n"
        f"  Relative difference: {abs((obj_LP - obj_PDL) / obj_LP)}"
    )

### Testing consistency of f_x(y) ###


In [17]:
import pyomo.environ as pyo
import numpy as np
import torch

class OptValueExtractor:
    def __init__(self):
        self.targets = []
    
    @property
    def opt_targets(self):
        return self.targets

    def get_avg_obj(self):
        return np.mean([target["obj"] for target in self.targets])

    def extract_values(self, model, solver):
        decision_vars = torch.tensor(self.extract_decision_variables(model))
        dual_vars_ineq, dual_vars_eq = self.extract_dual_vars(model, solver)
        dual_vars_ineq = torch.tensor(dual_vars_ineq)
        dual_vars_eq = torch.tensor(dual_vars_eq)
        ineq_slack = torch.tensor(self.extract_ineq_slack(model))
        eq_slack = torch.tensor(self.extract_eq_slack(model))
        obj_value = torch.tensor(model.obj())
        target = {"y": decision_vars, "mu": dual_vars_ineq, "lamb": dual_vars_eq, "ineq": ineq_slack, "eq": eq_slack, "obj": obj_value}
        self.targets.append(target)


    def extract_decision_variables(self, model):
        variables = [model.vGenProd, model.vLineFlow, model.vLossLoad, model.vGenInv]
        decision_vars = []
        for var in variables:
            if var.is_indexed():
                decision_vars += [var[idx].value for idx in var]
            else:
                # For scalar variables
                decision_vars += [var.value]
        
        return decision_vars

    def extract_dual_vars(self, model, solver):
        # Combine variables and inequality constraints into one list
        constraints = [
            model.eMaxProd, model.eLineFlowLB, model.eLineFlowUB, model.eRampingDown,
            model.eRampingUp, model.eGenProdPositive, model.eMissedDemandPositive, model.eMissedDemandLeqDemand, model.eGenInvPositive
        ] # First vLineFlow = lower bound. Second is upper bound.

        # Equality constraints
        eq_constraints = [model.eNodeBal]

        dual_vars_ineq = []
        dual_vars_eq = []

        # Process variables and inequality constraints
        for i, item in enumerate(constraints):
            if item.is_indexed():  # Indexed constraints or variables
                for idx in item:
                    if item[idx] in model.dual:
                        dual_vars_ineq.append(-model.dual[item[idx]])
                    else:
                        dual_vars_ineq.append(0)  # Append 0 if no dual exists, because it means the constraint is satisfied

            else:  # Scalar constraints or variables
                if item in model.dual:
                    dual_vars_ineq.append(-model.dual[item])
                else:
                    dual_vars_ineq.append(0)  # Append 0 if no dual exists, because it means the constraint is satisfied
            print("-"*20)

        # Process equality constraints
        for constr in eq_constraints:
            if constr.is_indexed():  # Handle indexed equality constraints
                for idx in constr:
                    if constr[idx] in model.dual:
                        dual_vars_eq.append(-model.dual[constr[idx]])
                    else:
                        dual_vars_eq.append(0)  # Append 0 if no dual exists
            else:  # Handle scalar equality constraints
                if constr in model.dual:
                    dual_vars_eq.append(-model.dual[constr])
                else:
                    dual_vars_eq.append(0)  # Append 0 if no dual exists

        return dual_vars_ineq, dual_vars_eq

    def extract_ineq_slack(self, model):
        ineq_constraints = [model.eMaxProd, model.eRampingDown, model.eRampingUp]
        variables = [model.vGenProd, model.vLineFlow, model.vLineFlow, model.vLossLoad, model.vLossLoad, model.vGenInv] # eGenProdPositive, eMissedDemandPositive, eMissedDemandLeqDemand eNumPowerGenerationUnitsPositive]
        ineq_slack = []
        for constr in ineq_constraints:
            for index in constr:
                ineq_slack.append(pyo.value(constr[index].body))
        for var in variables:
            if var.is_indexed():
                values = [var[idx].value for idx in var]
                ineq_slack += values
            else:
                # For scalar variables
                ineq_slack += [var.value]
        return ineq_slack
        
    def extract_eq_slack(self, model):
        eq_constraints = [model.eNodeBal]
        eq_slack = []
        for constr in eq_constraints:
            for index in constr:
                eq_slack.append(pyo.value(constr[index].body))
        
        return eq_slack

In [ ]:
model.dual.display()

dual : Direction=IMPORT, Datatype=FLOAT
    Key                            : Value
        eGenInvPositive[BEL,SunPV] :                 0.0
        eGenInvPositive[GER,SunPV] :                 0.0
        eGenInvPositive[NED,SunPV] :                 0.0
    eGenProdPositive[BEL,SunPV,10] :                 0.0
    eGenProdPositive[BEL,SunPV,11] :                 0.0
    eGenProdPositive[BEL,SunPV,12] :                 0.0
     eGenProdPositive[BEL,SunPV,1] :                 0.0
     eGenProdPositive[BEL,SunPV,2] :                 0.0
     eGenProdPositive[BEL,SunPV,3] :                 0.0
     eGenProdPositive[BEL,SunPV,4] :                 0.0
     eGenProdPositive[BEL,SunPV,5] :                 0.0
     eGenProdPositive[BEL,SunPV,6] :                 0.0
     eGenProdPositive[BEL,SunPV,7] :                 0.0
     eGenProdPositive[BEL,SunPV,8] :                 0.0
     eGenProdPositive[BEL,SunPV,9] :                 0.0
    eGenProdPositive[GER,SunPV,10] :                 0.0
    e

In [ ]:
extractor = OptValueExtractor()
for model in models:
    extractor.extract_values(model, solvers[0])

--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
-------------

In [ ]:
def calculate_dual_objective(mu, lamb, rhs_ineq, rhs_eq):
    """ max -mu^T d - lamb^T b

    Args:
        mu (_type_): _description_
        lamb (_type_): _description_
        rhs_ineq (_type_): _description_
        rhs_eq (_type_): _description_

    Returns:
        _type_: _description_
    """
    ineq_term = torch.sum(mu * rhs_ineq)
    
    eq_term = torch.sum(lamb * rhs_eq)

    return -(ineq_term + eq_term)

for idx, t_range in enumerate(T_ranges):

    # Inequalities
    rhs_ineq = []
    # 3.1b
    rhs_ineq += [0] * len(input_G) * SAMPLE_DURATION
    # 3.1d
    rhs_ineq += data.pImpCap_tensor.unsqueeze(0).expand(12, 3).T.flatten() # L*T
    # 3.1e
    rhs_ineq += data.pExpCap_tensor.unsqueeze(0).expand(12, 3).T.flatten() # L*T
    # 3.1f
    rhs_ineq += [0] * len(input_G) * (SAMPLE_DURATION - 1)
    # 3.1g
    rhs_ineq += [0] * len(input_G) * (SAMPLE_DURATION - 1)
    # 3.1h
    rhs_ineq += [0] * len(input_G) * SAMPLE_DURATION
    # 3.1i
    rhs_ineq += [0] * len(input_N) * SAMPLE_DURATION
    #3.1j
    rhs_ineq += data.pDemand_tensor[:, SAMPLE_DURATION*idx:SAMPLE_DURATION*(idx+1)].flatten()
    #3.1k
    rhs_ineq += [0] * len(input_G)

    # Equalities
    rhs_eq = []
    #3.1c
    rhs_eq += data.pDemand_tensor[:, SAMPLE_DURATION*idx:SAMPLE_DURATION*(idx+1)].flatten()

    mu = extractor.opt_targets[idx]["mu"]
    lamb = extractor.opt_targets[idx]["lamb"]

    y = extractor.opt_targets[idx]["y"].unsqueeze(0)

    dual_obj_val = calculate_dual_objective(mu, lamb, torch.tensor(rhs_ineq), torch.tensor(rhs_eq))
    assert abs(dual_obj_val - models[idx].obj()) <= 1e-6

In [ ]:
def calculate_dual_objective(gep_problem, X, y, mu, lamb):
    """
    Calculate the dual objective function for the given problem.

    Args:
        gep_problem: Instance of the GEPProblem class.
        X: Input tensor (batch of input parameters).
        y: Decision variables tensor (primal solution).
        mu: Dual variables for inequality constraints.
        lamb: Dual variables for equality constraints.

    Returns:
        Dual objective value for the batch.
    """
    # Compute inequality and equality constraints
    g_x_y = gep_problem.g(X, y)  # Inequality constraints
    h_x_y = gep_problem.h(X, y)  # Equality constraints

    ineq_term = torch.sum(mu * g_x_y, dim=1) 
    eq_term = torch.sum(lamb * h_x_y, dim=1)

    print(f"Ineq term: {ineq_term}")
    print(f"eq term: {eq_term}")

    # Compute the dual objective
    dual_obj = gep_problem.obj_fn(y) + ineq_term + eq_term  # Sum over constraints

    return dual_obj

x = all_X[0].unsqueeze(0)
y = extractor.opt_targets[0]["y"].unsqueeze(0)

g = data.ineq_resid(x, y)
h = data.eq_resid(x, y)
obj = data.obj_fn(y)
print(obj)

mu = extractor.opt_targets[0]["mu"]
lamb = extractor.opt_targets[0]["lamb"]

print(len(mu))

print(f"Dual objective: {calculate_dual_objective(data, x, y, mu, lamb).item()}")
print(f"Primal objective: {model.obj()}")

tensor([8.1865e+08])
285
Ineq term: tensor([0.])
eq term: tensor([1.5478e-09])
Dual objective: 818648032.282804
Primal objective: 818648032.2828041


In [ ]:
# eMaxProd, 3.1b:
# print(g[0][:36].tolist())
# print(mu[:36].tolist())
print(g.tolist())
print(mu.tolist())

print(h.tolist())
print(lamb.tolist())

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -28381.927787824792, -42572.891681737194, -57561.10073822893, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -71323.08656009697, -105344.32044648677, -124022.35518166499, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -24053.013750341364, -40406.52638453189, -41862.85891048555, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1724.6005319384499, -1497.256707782, 0.0, -5400.0, -5400.0, -5400.0, -5400.0, -5400.0, -5400.0, -5400.0, -5400.0, -5800.0, 0.0, 0.0, -1248.35441754176, -3000.0, -3000.0, -3000.0, -3000.0, -3000.0, -3000.0, -3000.0, -3000.0, -10000.0, 0.0, 0.0, 0.0, -4380.0, -4380.0, -4380.0, -4380.0, -4380.0, -4380.0, -4380.0, -4380.0, -4380.0, -2655.39946806155, -2882.743292218, -4380.0, -400.0, -400.0, -400.0, -400.0, -400.0, -400.0, -400.0, -400.0, 0.0, -5800.0, -5800.0, -4551.64558245824, -7000.0, -7000.0, -7000.0, -7000.0, -7000.0, -7000.0, -7000.0, -7000.0, 0.0, -10000.0, -10000.0, -10000.0, -31889.806503173924, -31889.806503173924, -3